Import all the necessary lib

In [1]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
from tqdm import tqdm
import time
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

#### Web_driver Initialization

In [44]:
chrome_options = Options()
service = Service('C:\\chromedriver\\chromedriver.exe')

In [45]:
driver = webdriver.Chrome(service=service, options=chrome_options)

In [46]:
driver.get('https://stock-pictures.netlify.app')

#### Auto Scroll

In [53]:
for scroll in tqdm(range(0,500000,1000)):
    driver.execute_script("window.scrollTo(0,"+str(scroll)+")")
    time.sleep(.5)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [04:21<00:00,  1.91it/s]


In [54]:
soup = bs(driver.page_source, 'html.parser')

#### Number of Images available

In [55]:
len(soup.find_all('div', class_ = 'container'))

9104

#### Scraping the Data and forming Data Frame 

In [83]:
data = []

for sp in tqdm(soup.find_all('div', class_ = 'container')):
    link = sp.find('img').get('src')                      # |-Remove-| part   
    tags = sp.find('span', class_ = 'tag-color').text[7:] # "Tags - "Clouds, Sky, Atmosphere, Blue Sky
    
    likes    = int(sp.find('div',class_ = 'likes-comments').find_all('span')[0].text[:-6])
    comments = int(sp.find('div',class_ = 'likes-comments').find_all('span')[1].text[:-9])
    
    #print(sp.find('div',class_ = 'tags').text[7:].strip())
    #break
    
    data.append([link, tags, likes, comments])

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9104/9104 [00:00<00:00, 12650.13it/s]


In [84]:
data

[['https://cdn.pixabay.com/photo/2022/03/06/05/30/clouds-7050884__480.jpg',
  'Clouds, Sky, Atmosphere, Blue Sky',
  196,
  55],
 ['https://cdn.pixabay.com/photo/2022/04/07/11/45/bird-7117346__340.jpg',
  'Bird, Ornithology, Hummingbird',
  76,
  20],
 ['https://cdn.pixabay.com/photo/2022/02/28/15/28/sea-7039471__340.jpg',
  'Sea, Rainbow, Rainfall, Subtropical',
  282,
  106],
 ['https://cdn.pixabay.com/photo/2022/04/04/02/52/cherry-blossoms-7110279__340.jpg',
  'Cherry Blossoms, Road, Japan, Sakura',
  42,
  11],
 ['https://cdn.pixabay.com/photo/2022/04/09/18/06/cape-marguerite-7121992__340.jpg',
  'Cape Marguerite, Flower, Plant',
  39,
  15],
 ['https://cdn.pixabay.com/photo/2021/11/13/23/06/tree-6792528__340.jpg',
  'Relaxing Rest Under The Tree',
  522,
  108],
 ['https://cdn.pixabay.com/photo/2022/04/07/02/56/cottontail-rabbit-7116707__340.jpg',
  'Cottontail Rabbit, Wild Rabbit, Grass',
  51,
  10],
 ['https://cdn.pixabay.com/photo/2022/03/19/21/11/crocus-7079550__340.jpg',
  '

In [85]:
data = pd.DataFrame(data, columns = ['Link', 'Tags', 'Likes', 'Comments'])
data

,Link,Tags,Likes,Comments
0,https://cdn.pixabay.com/photo/2022/03/06/05/30...,"Clouds, Sky, Atmosphere, Blue Sky",196,55
1,https://cdn.pixabay.com/photo/2022/04/07/11/45...,"Bird, Ornithology, Hummingbird",76,20
2,https://cdn.pixabay.com/photo/2022/02/28/15/28...,"Sea, Rainbow, Rainfall, Subtropical",282,106
3,https://cdn.pixabay.com/photo/2022/04/04/02/52...,"Cherry Blossoms, Road, Japan, Sakura",42,11
4,https://cdn.pixabay.com/photo/2022/04/09/18/06...,"Cape Marguerite, Flower, Plant",39,15
...,...,...,...,...
9099,https://cdn.pixabay.com/photo/2018/01/08/18/51...,"Pear, Fruit, Food, Background, Healthy",75,11
9100,https://cdn.pixabay.com/photo/2021/09/22/01/42...,"Ants, Insects, Plants, Flowers, Animal",13,8
9101,https://cdn.pixabay.com/photo/2022/01/09/15/58...,"Flower, Flora, Garden, Nature",0,0
9102,https://cdn.pixabay.com/photo/2021/08/10/11/59...,"Kara Dag Mountain, Resort Village, Town",6,2


In [88]:
data.to_csv('ImageScraping.csv', index = False)

#### Downloading the Images and Storing in a Folder

In [89]:
import requests

In [98]:
loc =[]

for i in tqdm(data['Link']):
    res = requests.get(i)

    #print(str(i)+"\t"+str(i.split('/')))
    #break
    
    #os.mkdir('Collection')
    
    # https://cdn.pixabay.com/photo/2022/03/06/05/30/clouds-7050884__480.jpg	
    # ['https:', '', 'cdn.pixabay.com', 'photo', '2022', '03', '06', '05', '30', 'clouds-7050884__480.jpg']
    # clouds-7050884__480.jpg ---> To get
    
    fd = open('Collection/'+i.split('/')[-1],'wb') #Selecting Directory and name of file
    fd.write(res.content) # Writing the data in sected directory with the given name
    fd.close()
    
    loc.append('Collection/'+i.split('/')[-1])

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9104/9104 [36:51<00:00,  4.12it/s]


In [99]:
data['Location'] = loc

#### Creating DataFrame and store it as .csv file

In [10]:
data.to_csv('ImageScraping_2.csv', index =False)

In [11]:
data

,Link,Tags,Likes,Comments,Location
0,https://cdn.pixabay.com/photo/2022/03/06/05/30...,"Clouds, Sky, Atmosphere, Blue Sky",196,55,Collection/clouds-7050884__480.jpg
1,https://cdn.pixabay.com/photo/2022/04/07/11/45...,"Bird, Ornithology, Hummingbird",76,20,Collection/bird-7117346__340.jpg
2,https://cdn.pixabay.com/photo/2022/02/28/15/28...,"Sea, Rainbow, Rainfall, Subtropical",282,106,Collection/sea-7039471__340.jpg
3,https://cdn.pixabay.com/photo/2022/04/04/02/52...,"Cherry Blossoms, Road, Japan, Sakura",42,11,Collection/cherry-blossoms-7110279__340.jpg
4,https://cdn.pixabay.com/photo/2022/04/09/18/06...,"Cape Marguerite, Flower, Plant",39,15,Collection/cape-marguerite-7121992__340.jpg
...,...,...,...,...,...
9088,https://cdn.pixabay.com/photo/2018/01/08/18/51...,"Pear, Fruit, Food, Background, Healthy",75,11,Collection/pear-3070020__340.png
9089,https://cdn.pixabay.com/photo/2021/09/22/01/42...,"Ants, Insects, Plants, Flowers, Animal",13,8,Collection/ants-6645412__480.jpg
9090,https://cdn.pixabay.com/photo/2022/01/09/15/58...,"Flower, Flora, Garden, Nature",0,0,Collection/flower-6926320__340.jpg
9091,https://cdn.pixabay.com/photo/2021/08/10/11/59...,"Kara Dag Mountain, Resort Village, Town",6,2,Collection/kara-dag-mountain-6535841__340.jpg


#### Checking for NULL values

In [12]:
data.isnull().sum()

Link        0
Tags        0
Likes       0
Comments    0
Location    0
dtype: int64

#### Droping NULL Values if found

In [108]:
data.dropna(inplace = True)

In [2]:
data = pd.read_csv('ImageScraping_2.csv')

In [20]:
tag = set()
for i in data['Tags']:
    for j in i.split(','):
        tag.add(j)

In [21]:
tag.remove('')

In [22]:
tag

{' Nara Park',
 ' Gourds',
 ' Krad',
 'Hudson River',
 ' Red Keys',
 ' Satellite Image',
 ' Mist',
 'Wave',
 " Men'S Suit",
 ' Projector',
 ' Stormy',
 ' Bird Feeder',
 ' Summit',
 'Judge',
 ' unweit von Chemnitz an der B174 Richtung Marienberg.',
 ' Evening',
 ' Sunset',
 'Great Cormorant',
 'Town',
 ' Light Bulb',
 ' Decor',
 'Sunlight',
 ' Prayer',
 ' travel',
 ' Flavoring',
 ' Signature',
 ' Motor',
 ' Spiral',
 ' Mix Media',
 ' Soil',
 ' Era',
 'assorted color smiley illustration',
 ' Salad Platter',
 ' Rots',
 ' Living Room',
 ' Wedding Day',
 'Still',
 ' Rose Haw',
 'Temple',
 'Maze',
 ' Floorball',
 'Grocery',
 ' Blue Star',
 ' Balcony',
 'Metatron Cube',
 'Excavator',
 ' Layout',
 ' Thunder',
 'Clown',
 'This is a hospital ship',
 ' Wave',
 ' To North',
 ' Videographer',
 ' Deer',
 ' Spices',
 'Party Lights',
 'Solar',
 ' Corridor',
 'Globe',
 ' Peanut Kernels',
 ' in French) convent in Toulouse (France) has become a museum.',
 'Cough',
 'Cornavirus',
 'Tea Lights',
 'Pigmywee

#### Creating Folders for each type of tag and storing imgaes corresponding to that tag

In [23]:
for i in tqdm(tag):
    try :
        os.mkdir('dataset/'+(i.strip()))
    except:
        pass

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9610/9610 [00:00<00:00, 21711.51it/s]


#### Coping the Images collected earlier to ther matching tags 

In [48]:
import shutil

In [36]:
error_incr = 0
for tags, img in tqdm(zip(data['Tags'],data['Location'])):
    #print(tags)
    #print(img)
    src = img

    tag_dir = ['dataset/' + tag.strip() + '/' for tag in tags.split(',')]
# Out -->  ['dataset/Family/', 'dataset/Mother/', 'dataset/Daughter/', 'dataset/Outdoors/', 'dataset/Hugs/']

    for tag_loc in tag_dir:
    # Out --> dataset/Family/
        
        '''if not os.path.exists(tag_loc):
            os.mkdir(tag_loc)
            '''
            #     dataset/Family/ + family-4447538__340.jpg  
        
        des = os.path.join(tag_loc, os.path.basename(src))
        try:
            copy(src, des)
        except Exception as e:
            pass

9093it [00:13, 694.44it/s]


In [37]:
len(os.listdir('dataset'))

7970

#### Calculating Number of Images within the Tag folders

In [39]:
folder_ = []
freq    = []

for folder in tqdm(os.listdir('dataset')):
    
    try:
        freq.append(len(os.listdir('Dataset/' + folder)))
        folder_.append(folder)

    except:
        pass
print(len(folder_),len(freq))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7970/7970 [00:00<00:00, 37073.56it/s]

7967 7967


#### Top 10 folders with most number of Images

In [40]:
df_ = pd.DataFrame()

df_['folder'] = folder_
df_['freq']   = freq

df_.sort_values(by = 'freq', ascending = False).head(10)

,folder,freq
4747,nature,838
226,Animal,608
2744,Flower,548
727,Bird,517
2760,Flowers,505
6096,Sea,305
5348,Plant,283
6777,Sunset,275
2811,Forest,261
3989,Landscape,242


#### 10 Least folders with images

In [179]:
df_.sort_values(by = 'freq', ascending = True).head(10)

,folder,freq
2282,Each candle is a soul,0
2004,Das versteckte Haus,0
2048,Der Sonne entgegen,0
2086,die hinauf führt,0
3657,In einem kleinen Städtchen,0
2559,Fantasmagòrica vista de los Reyes Magos de Ori...,0
2387,emergiendo,0
4686,Music Wallpaper,1
4684,Music Notes,1
4683,Music Melody,1


#### Avg no.of images in each folder

In [41]:
df_['freq'].mean()

4.70653947533576

### Removing Folders with less than 20 images

In [63]:
remove = df_[df_['freq']<20]['folder']
remove

0       &quot;All we are saying is give peace a chance...
1       &quot;Gray wagtail&quot; found in rivers and m...
2       (Sturnus vulgaris)​ es una especie de ave pass...
3       1891. In 1906 she was en route from Peru to Ge...
4                                                    1925
                              ...                        
7962                               Église Dol de Bretagne
7963                                               сакура
7964                          „Ich schau dir in die Augen
7965                                 奈良公園の鹿です、人間と共存しています。
7966                      👉 If you find my content useful
Name: folder, Length: 7685, dtype: object

In [68]:
for rem in tqdm(remove):
    print(rem)
    src = 'dataset/' + rem
    dst = 'Temp/' + rem
    try:
        shutil.move(src, dst)
    except:
        pass

  7%|█████████████▊                                                                                                                                                                          | 575/7685 [00:00<00:01, 5018.04it/s]

&quot;All we are saying is give peace a chance.&quot; (John Lennon)
&quot;Gray wagtail&quot; found in rivers and mountain streams in the subalpine zone
(Sturnus vulgaris)​ es una especie de ave passeriforme de la familia Sturnidae nativa del Paleártico. En Argentina fue introducido hace unos años y hoy se reproduce abundantemente en el sudeste de la Provincia de Buenos Aires
1891. In 1906 she was en route from Peru to Geelong
1925
1950S
2020
2021
2022
3D
3D Animation
3D Render
4
4 spreads and back cover of the magazine. Name of the editorial &quot;The Monochrome Chapter&quot;
4 Weihnachtskerzen
4 X 4
4-Cyl Boxer
404
5G
5X8 Notepad
776m) is the highest mountain in our country. I took this photo from Mitsutouge-sanso hut
955m in the Italian Alps
A
A Book
a consequence of global warming
A Couple Of Poppy Flowers
A Drawer
A Fishing
A Handful Of
A Heart
A Hit
A Notice
A Princess
a route between Cain and Poncebos of about 12 km that connects both towns
A Smile
A splash of colour in the grey


 22%|████████████████████████████████████████▋                                                                                                                                              | 1711/7685 [00:00<00:01, 5381.16it/s]

Cannabis
Cannabis Oil
Cannabis Online
Cannabis Vera
Cannon
Canoe
Cans
Cantuccini
Canvas
Canyon
Cape Arkona
Cape Gooseberry
Cape Greco
Cape Leadwort
Cape Marguerite
Cape Periwinkle
Cape Town
Capital
Cappadocia
Cappuccino
Capsella
Capsicum
Captive Balloon
Captivity
Car Dashboard
Car Racing
Car Radios
Car Wash
Caracas
Carafe
Caravan
Carbon
Carbon Dioxide
Card
Card Love
Card Payment
Cardinal
Cardinals
Cards
Care
Career
Cares river route
Cargo
Cargo Plane
Cargo Ship
Caribbean
Carinthia
Carnation
Carneval
Carnival
Carnivore
Carnivores
Caro aus ganz anderer Perspektive
Carolina Wren
Carousel
Carousel Horse
Carp
Carpenter
Carpenter Bee
Carpet
Carpet Of Flowers
Carriage
Carrots
Cars
Cart
Cartology
Carton
Cartoon
Cartoon Family
Cartoons
Casablanca
Casares
Cascade
Cash
Cashews
Casino
Cassette
Cassette Tape
Cassette Tapes
Cassowary
Cassowary Feet
Cast
Castel San Pietro
Castelvecchio
Castle Hluboká
Castle Marcus
Castle Yard
Casual Shoes
Cat Eyes
Cat Face
Cat Mother
Cat Portrait
Cat'S Eyes
Cat-Bear


 38%|█████████████████████████████████████████████████████████████████████▋                                                                                                                 | 2929/7685 [00:00<00:00, 5544.79it/s]

...

Olive Fruits
Olive Oil
Olive Tree
Olives
Olympic Pool
Olympic Stadium
Omelette
Omikuji indicate one’s overall fortune and suggest what should be done about such as marriage
Omnivore
Omr
On
On peut sourire et sourire et pourtant être un scélérat
One
One Person
Onion
Onions
Online
Online Games
Online Payment
Online Sales
Ontario
Open
Open Book
Open Mouth
Open Sign
Opera
Operation
Opinion Poll
Opium Poppy
Opposite
Optics
Orange
Orange Beak
Orange Color
Orange Daisy
Orange Flower
Orange Flowers
Orange Leaves
Orange Rose
Orange Roses
Orange Slices
Orange Trees
Orange-Breasted Sunbird
Oranges
Orangutan
Orchestra
Orchid
Orchid Flower
Orchids
Ore Mining
Organ
Oriental
Oriental Garden Lizard
Oriental Magpie-Robin
Origami
Original
Orion Nebula
Ornament
Ornamental Cherry
Ornamental Grass
Ornamental Lily
Ornamental Plants
Ornaments
Orphan
Orpine
Ortahisar
Orthodontics
Orthodox Church
Oslo
Osprey
Ostrich
Ostrich Dad and chick
Otter
Our Lady
Out
Outdoor
Outdoor Sports
Outer Space
Outlook
Outside
Ove

 85%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉                           | 6550/7685 [00:01<00:00, 5682.66it/s]

Seashore
Seaside
Seaside Daisy
Seasonal
Seasoning
Seasons
Seat
Seattle
Seawall
Secluded
Secret
Secured Slide Bolt and Combination Lock
Security
Security Camera
Seductive
See
Seed
Seed Pod
Seedling
Seedlings
Seeds
Seek
Seem
Segovia
Seifenblasen auf Flüssigkeit farbig
Self Care
Self Confidence
Self Portrait
Self-Quarantine
Self-Test
Selfie
Seljalandsfoss
Seller
Selling Online
Selva Di Cadore
Semi-Trailer
Senior
Sensual
Seo
Seoul
Sequin Fabric
Sequins
Serval
Server
Sesame Oil
Set
Set of abstract ocean scenery
Setaria Pumila
Sever
Several
Seville
Sew
Sewing
Sewing Machine
Sewing Thread
Sex
Sexual
Sexuality
Sexy
Sexy Girl
Shabby Chic
Shack
Shade
Shades
Shades Of Color
Shadow
Shadows
Shake Hands
Shaking Hands
Shallots
shallow focus photo of flowers
shallow focus photo of pink flower
Shallows
Shanghai
Shapes
Shark
Sharks
Sharp
Shatter
Shavings
Shawarma
Shea Butter
Sheep
Sheeps
Sheet
Sheet Music
Sheets
Sheikh Zayed Mosque
Shelducks
Shelf
Shell
Shellfish
Shells
Shelter
Sheltie
Shelves
Shenzhen


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7685/7685 [00:01<00:00, 5599.48it/s]

Unit
United States
United States Flag
Universe
University
University Library
Unknown
Unlock
unsere Bonny riecht und schmeckt den Frühling!
unsere Maine-Coon-Katze
Unshelled Almonds
Unsure
unweit von Chemnitz an der B174 Richtung Marienberg
Upholstery
Upset
Upwards
Ural Owl
Urban Areas
Urban Design
Ursus Arctos
Urtica
Us
Us Car
Us Dollar
Us Truck
Usa
Uscgc Eagle
Used
Utah
Utensils
Utv
Uzbekistan
V A Waterfront
Vacation
Vacations
Vaccination
Vaccination Certificate
Vaccine
Valais
Valencia
Valentin
Valentine
Valentine'S
Valentine'S Day
Valentinstag
Valle Verzasca
Valley
Valley Of Death
Valmue
Valravn
Vampire
Van
Van Cat
Vanda Mokara
Vanessa Atalanta
Vapor
Variety
Varinasia
Varnish
Vase
Vault
Vector Notepad
Vegetable Juices
Vegetables Landscape
Vegetarian
Vehicles
Veil
Veins
Velvet
Velvet Foot
Veneto
Venezuela
Venice
Venice Carnival
Ventura
Venus
Verona
Veronica Persica
Versailles
verse appels
Verzasca Valley
Vespa
Vessel
Vessels
Vial
Victoria Crowned Pigeon
Victoria Falls
Victoria. The wa

#### tags with 20+ imges

In [69]:
len(os.listdir('dataset'))

285

#### tags <20 imh 

In [70]:
len(os.listdir('Temp'))

7685